# Extract all tables in `sales` schema

## Environment configuration

In [1]:
%load_ext autoreload
%autoreload 2

Import necessary packages

In [8]:
dbutils.widgets.dropdown("environment", "dev", ["dev", "prod"], "Environment")
environment = dbutils.widgets.get("environment")
environment

'dev'

In [18]:
import json
from pyspark.sql.functions import lit
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

Open json file with control variables of the task.

In [19]:
with open('tables_mapping.json', 'r') as f:
    config = json.load(f)

Define constants

In [20]:
SCHEMA = config["general"]["schema"]
CATALOG = f'{config["general"]["user"]}_raw'
DB_PASSWORD = dbutils.secrets.get(scope="antonio_junior_adw", key="pswd_mssql")
DB_HOST = dbutils.secrets.get(scope="antonio_junior_adw", key="ip_mssql")
DB_PORT = dbutils.secrets.get(scope="antonio_junior_adw", key="port_mssql")
DB_USER = config["general"]["db_user"]
DATABASE =  config["general"]["database"]
TABLES_LIST = list(config["tables"].keys())
environment = dbutils.configuration.get("environment")

The time window for data extraction defaults to the last seven days.

In [ ]:
last_modified_date = datetime.today() - timedelta(days=7)
last_modified_date = last_modified_date.strftime('%Y-%m-%d 00:00:00')
last_modified_date = "2007-01-01 00:00:00"

Defines the JDBC connection string for the database.

In [21]:
jdbc_url = f"jdbc:sqlserver://{DB_HOST}:{DB_PORT};databaseName={DATABASE};encrypt=true;trustServerCertificate=true;"

connection_properties = {
    "user": DB_USER, 
    "password": DB_PASSWORD,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

## Extraction steps

Defines a function to extract the desired data from each table.

In [7]:
def extract_data(table_name: str) -> None:
    """
    A function to extract data from a table.

    Args:
        table_name (str): The table name in the database.
    """
    # Define the destination path
    delta_table_name = f"{CATALOG}.{SCHEMA}.raw_{environment}_{table_name}"
    print("Extracting data to:", table_name)

    # Query to select data accoding with the ModifiedDate column
    query = f"(SELECT * FROM {SCHEMA}.{table_name} WHERE ModifiedDate >= '{last_modified_date}') AS subquery"

    # Read data from the database
    df = spark.read.jdbc(
        url=jdbc_url,
        table=query,
        properties=connection_properties
    )

    # Insert a column into the dataframe with the extraction date
    df = df.withColumn("extract_date", lit(datetime.today()))

    # Save the df into the destination path
    df.write.mode("overwrite").format("delta").saveAsTable(delta_table_name)

The extraction runs in parallel, processing all tables listed in `tables_mapping.json.

In [10]:
with ThreadPoolExecutor(max_workers=8) as executor:
    executor.map(extract_data, TABLES_LIST)

Extracting data to: CountryRegionCurrency
Extracting data to: Currency
Extracting data to: CurrencyRate
Extracting data to: Customer
Extracting data to: PersonCreditCard
Extracting data to: SalesOrderDetail
Extracting data to: SalesOrderHeader
Extracting data to: SalesOrderHeaderSalesReason
Extracting data to: SalesPerson
Extracting data to: SalesPersonQuotaHistory
Extracting data to: SalesReason
Extracting data to: SalesTaxRate
Extracting data to: SalesTerritory
Extracting data to: SalesTerritoryHistory
Extracting data to: ShoppingCartItem
Extracting data to: SpecialOffer
Extracting data to: SpecialOfferProduct
Extracting data to: Store
